# Coalition-Finding Algorithmus

Dieses Notebook demonstriert die Nutzung des Algorithmus `subset_finding`, um Koalitionen von Features mit dem höchsten bzw. niedrigsten aggregierten Interaktionswert zu identifizieren.

Als Grundlage dienen Interaktionswerte, die auf einem erklärbaren Modell basieren und mit Hilfe von Indizes wie `k-SII`, `STII` oder `FSII` berechnet wurden.

Der zugrundeliegende Algorithmus basiert auf einer heuristischen Beam Search, die durch gezielte Vorauswahl vielversprechender Teilkoalitionen eine effiziente Suche in großen Feature-Räumen ermöglicht.





## Vorbereitung: Imports

Wir beginnen mit dem Import aller benötigten Module für Modellierung, Datenaufbereitung und den Koalitionsfindungs-Algorithmus.



In [39]:
# -------------------------------------------------------------
# Warnungen unterdrücken für saubere Ausgabe im Notebook
# -------------------------------------------------------------
import warnings

# Warnung bei .to_numpy() in Kombination mit sklearn-Modellen
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but RandomForestRegressor was fitted with feature names",
    category=UserWarning,
)

# Warnung bei zu kleinem Hintergrunddatensatz für Explainer
warnings.filterwarnings(
    "ignore",
    message="The sample size is larger than the number of data points in the background set.*",
    category=UserWarning,
)

# -------------------------------------------------------------
# Standardbibliotheken und Framework-Imports
# -------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# shapiq: Erklärbarkeit und Beispiel-Datensatz
from shapiq import Explainer
from shapiq.datasets import load_bike_sharing



## Datensatz und Regressionsmodell

In diesem Beispiel verwenden wir den `bike_sharing`-Datensatz, der stündliche Fahrradverleihzahlen und zugehörige Kontextinformationen wie Temperatur, Luftfeuchtigkeit oder Uhrzeit enthält.

Wir trainieren ein Regressionsmodell, das die Ausleihzahl vorhersagt. Standardmäßig verwenden wir hier einen Random Forest Regressor – andere Alternativen wären z. B. `LinearRegression`, `KNeighborsRegressor` oder `DecisionTreeRegressor`.

Anschließend berechnen wir Interaktionswerte mithilfe des `k-SII`-Index.
Diese Werte dienen als Grundlage für die nachfolgende Koalitionsanalyse.


In [34]:
# Bike-Sharing-Daten laden
X, y = load_bike_sharing()

# Wir wählen 100 zufällige Datenpunkte für die spätere Koalitionsanalyse
X = X.sample(n=100, random_state=42)
y = y.loc[X.index]

# Trainingsdaten aufteilen und Modell trainieren
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=1)
model = RandomForestRegressor().fit(X_train, y_train)

## Interaktionswerte berechnen

Um sinnvolle Koalitionen identifizieren zu können, benötigen wir sogenannte Interaktionswerte.
Diese quantifizieren den Beitrag einzelner oder kombinierter Features zur Vorhersage eines Modells.

Wir verwenden dazu den `Explainer` aus der `shapiq`-Bibliothek mit dem Index `k-SII` (Shapley Interaction Index).
Dabei legen wir die maximale Interaktionsordnung (`max_order`) auf 3 fest – d. h. es werden Einzel-, Paar- und Dreifachinteraktionen berücksichtigt.

Die Erklärung erfolgt für einen einzelnen Datenpunkt aus dem Trainingsdatensatz.


In [35]:
from shapiq import Explainer

explainer = Explainer(
    model=model.predict,
    data=X_train.to_numpy(),
    index="k-SII",
    max_order=3,
)

# Wir erklären den ersten Datenpunkt aus dem Trainingsset
x0 = X_train.to_numpy()[0]
interaction_values = explainer.explain(x0, budget=16)
interaction_values

InteractionValues(
    index=k-SII, max_order=3, min_order=0, estimated=True, estimation_budget=16,
    n_players=12, baseline_value=191.13187499999998
)

## Koalitionsfindung mit Beam Search (Theorie)

Der Algorithmus `subset_finding` nutzt eine heuristische Beam Search zur effizienten Koalitionssuche.

Ziel ist es, unter allen möglichen Koalitionen der Größe $l$ diejenigen zu finden, die den höchsten oder niedrigsten Gesamtbeitrag zur Vorhersage eines Modells leisten – gemessen an den sogenannten Interaktionswerten.


### Wie wird eine Koalition bewertet?

Jede Koalition $S$ besteht aus einer Teilmenge von Features, z. B. $S = \{A, B\}$.
Zur Bewertung wird der gesamte Einfluss dieser Feature-Kombination auf das Modell aggregiert:

$$
\hat{\nu}_e(S) = e_0 + \sum_{i \in S} e_{\{i\}} + \sum_{\substack{i,j \in S \\ i < j}} e_{\{i,j\}} + \sum_{\substack{T \subseteq S \\ |T| > 2}} e_T
$$

Dabei gilt:
- $e_0$: Basiswert ohne Features (leere Koalition)
- $e_{\{i\}}$: Beitrag des einzelnen Features $i$
- $e_{\{i,j\}}$: Beitrag des Zusammenspiels von zwei Features
- $e_T$: höherwertige Interaktionen (drei oder mehr)

Je höher der Wert $\hat{\nu}_e(S)$, desto stärker der Einfluss der Koalition auf die Vorhersage (positiv oder negativ).

### Beispiel

Gegeben sei eine Feature-Menge $N = \{A, B, C\}$ mit folgenden Interaktionen:

- $e_{\{A\}} = 1.0$, $e_{\{B\}} = -0.5$, $e_{\{A,B\}} = 0.3$
- $e_{\{C,B\}} = -0.2$, $e_{\{C\}} = -0.1$, $e_{\{A,C\}} = 0.2$, $e_{\{A,B,C\}} = 0.6$, $e_0 = 0.0$

Dann ergibt sich für die Koalition $S = \{A, B\}$:

$$
\hat{\nu}_e(\{A,B\}) = 0.0 + 1.0 - 0.5 + 0.3 = 0.8
$$

### Wie findet der Algorithmus solche Koalitionen effizient?

Statt alle möglichen Kombinationen zu prüfen, was bei vielen Features sehr aufwändig wäre, geht der Algorithmus schrittweise vor:

- Zuerst werden alle einzelnen Features als Startpunkte betrachtet
- In mehreren Runden werden die vielversprechendsten Kandidaten um je ein weiteres Feature erweitert
- In jeder Runde bleiben nur die bestbewerteten Kombinationen erhalten, alle anderen werden verworfen

Der Algorithmus besteht dabei aus zwei klar getrennten Phasen:

1. **Approximationsphase**:
   In dieser Phase erfolgt die Bewertung der Koalitionen heuristisch.
   Höherwertige Interaktionen werden vorab anteilig auf die Einzelspieler verteilt.
   So kann der Algorithmus Koalitionen effizient vergleichen, ohne die vollständige Formel auszuwerten.

2. **Genauigkeitsphase**:
   Sobald die Anzahl der Kandidaten unter eine festgelegte Schwelle sinkt (die sogenannte Threshold-Beam-Breite),
   wird die Bewertung auf Basis der exakten Bewertungsfunktion durchgeführt – also unter Berücksichtigung aller vorhandenen Hyperkanten.

Auf diese Weise lassen sich Koalitionen mit extremen Interaktionswerten effizient und präzise identifizieren.



## Koalitionsfindung mit Beam Search (Praxis)

Wir wenden nun den Algorithmus `subset_finding` auf die berechneten Interaktionswerte an.
Gesucht wird eine Koalition fester Größe (hier: $l = 3$) mit maximalem bzw. minimalem aggregierten Interaktionswert gemäß der zuvor erklärten Bewertungsfunktion.


In [40]:
# Import des Koalitionsfindungs-Algorithmus
from shapiq_student.subset_finding import subset_finding

# Suche nach Koalitionen der Größe 3
beam_result = subset_finding(interaction_values=interaction_values, max_size=3)
beam_result

InteractionValues(
    index=k-SII, max_order=3, min_order=3, estimated=True, estimation_budget=None,
    n_players=12, baseline_value=191.13187499999998
)

## Analyse der gefundenen Koalitionen

Die Funktion `subset_finding` liefert zwei Koalitionen:

- `s_max`: Die Koalition der Größe `l`, die den höchsten aggregierten Interaktionswert erzielt
- `s_min`: Die Koalition mit dem niedrigsten Wert

Zusätzlich gibt sie auch die zugehörigen aggregierten Interaktionswerte `v_max` und `v_min` zurück.

Im Folgenden untersuchen wir die Koalitionen genauer – sowohl in numerischer Form (Index) als auch lesbar mit Spaltennamen.


In [43]:
# Rohausgabe
print("Koalition mit höchstem, aggregierten Interaktionswert (Index):", beam_result.s_max)
print("Koalition mit niedrigstem, aggregierten Interaktionswert (Index):", beam_result.s_min)
print("Interaktionswert der Koalition mit höchstem, aggregierten Interaktionswert:", beam_result.v_max)
print("Interaktionswert der Koalition mit niedrigstem, aggregierten Interaktionswert:", beam_result.v_min)

# Lesbare Darstellung mithilfe der Spaltennamen
columns = X_train.columns


def readable(S: set[int]) -> set[str]:
    return {columns[i] for i in S}


print("Koalition mit höchstem, aggregierten Interaktionswert (Namen):", readable(beam_result.s_max))
print("Koalition mit niedrigstem, aggregierten Interaktionswert (Namen):", readable(beam_result.s_min))

Koalition mit höchstem, aggregierten Interaktionswert (Index): {8, 9, 5}
Koalition mit niedrigstem, aggregierten Interaktionswert (Index): {0, 1, 2}
Interaktionswert der Koalition mit höchstem, aggregierten Interaktionswert: 195.56869930600533
Interaktionswert der Koalition mit niedrigstem, aggregierten Interaktionswert: 42.10465199174142
Koalition mit höchstem, aggregierten Interaktionswert (Namen): {'weekday', 'workingday', 'year'}
Koalition mit niedrigstem, aggregierten Interaktionswert (Namen): {'feel_temp', 'temp', 'hour'}


## Fazit

Der heuristische Beam Search Algorithmus ermöglicht eine effiziente Identifikation von Koalitionen mit hohem oder niedrigem Einfluss auf eine Modellvorhersage – selbst bei großen Feature-Mengen.

Durch die Kombination aus Approximations- und Genauigkeitsphase werden vielversprechende Koalitionen gezielt ausgewählt und präzise bewertet.
So lässt sich eine starke Reduktion des Suchraums erzielen, ohne auf gute Lösungen verzichten zu müssen.

Der Algorithmus lässt sich flexibel auf Interaktionen höherer Ordnung anwenden und stellt eine leistungsfähige Erweiterung
zur Analyse modellbasierter Erklärungen dar.
